In [2]:
import os
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
from PIL import Image
from glob import glob
import random
import nia22
import json
import cv2
import matplotlib.pyplot as plt 

Label  
FORMAT:  
Face left right origin 3DGaze 2DGaze

In [117]:
def cut_face(img, eye, factor = 1.5):
    # factor = 1.5: 눈 사이 거리의 +/- 1.5배를 얼굴로 봄
    both_eyes = np.array(eye.l_eyelid['points']+eye.r_eyelid['points'])

    center = both_eyes.mean(axis=0).astype(int)

    ll = np.array(eye.l_eyelid['points'])
    rr = np.array(eye.r_eyelid['points'])

    lc = np.mean(ll, axis=0)
    rc = np.mean(rr, axis=0)

    d_eyes = (np.sqrt(np.sum(np.square(lc - rc))) * factor).astype(int)

    # make sure square and within the image boundary
    lower = center - d_eyes

    lower[lower < 0] = 0

    upper = np.array([min((1920, lower[0] + 2*d_eyes)), min((1080, lower[1] + 2*d_eyes))])

    aside = min((upper - lower))

    upper[0] = lower[0] + aside
    upper[1] = lower[1] + aside
    
    return img[lower[1]:upper[1], lower[0]:upper[0]]

얼굴 crop  

얼굴 = 양눈 중앙에서 위-아래로 양눈 거리 * n 만큼

양 눈의 dx보다 dy가 더 큰 경우 = 이미지가 옆으로 누워있음. 


In [132]:
def calculate_gaze3d(anno):
    """실제론 좀 더 복잡한 3D 변환이 필요함.
    """
    return anno["Annotations"]["pose"]["gaze"]

def calculate_gaze2d(anno):
    """실제론 좀 더 복잡한 3D 변환이 필요함.
    """
    return anno["Annotations"]["pose"]["head"][1:]

In [156]:
base_dir = "/home/hoseung/Work/NIA/onecycle/l2cs/"
wdir = "train"

json_dir = "/home/hoseung/Work/NIA/onecycle_final/json/"
png_dir = "/home/hoseung/Work/NIA/onecycle_final/png/"


# output dir
label_dir = "./datasets/nia2022/Label/"
image_dir = "./datasets/nia2022/Image/"


os.makedirs(label_dir, exist_ok=True)
os.makedirs(image_dir, exist_ok=True)
os.makedirs(image_dir+"test/", exist_ok=True)
os.makedirs(image_dir+"train/", exist_ok=True)
os.makedirs(image_dir+"val/", exist_ok=True)

cnt = 0

In [134]:
jlist = glob(json_dir+"*.json")

random.shuffle(jlist)

# split the list of 원천데이터 8:1:1
nj = len(jlist)
ntest = nvalid = int(0.1*nj)

jtest = jlist[:ntest]
jvalid = jlist[ntest:ntest+nvalid]
jtrain = jlist[ntest+nvalid:]

In [135]:
print(len(jtest), len(jvalid), len(jtrain))

304 304 2437


In [155]:
image_dir

'./datasets/nia2022/Image/'

In [172]:
dtype = "test"

def make_dataset(fnlist, dtype, cnt=0):
    dir_out = f"{dtype}/"
    os.makedirs(image_dir+dir_out+"Face", exist_ok=True)
    os.makedirs(image_dir+dir_out+"Left", exist_ok=True)
    os.makedirs(image_dir+dir_out+"Right", exist_ok=True)

    f = open(image_dir.replace("Image", "Label") + f"{dtype}.label", "w")
    f.write("Face Left Right Origin 3DGaze 2DGaze")

    # load png and anno
    for fn_j in fnlist:
        try:
            anno = json.load(open(fn_j))
            eye = nia22.eyes.Eye(anno["Annotations"]["image"]["annotations"])

            # read png
            fn_png = fn_j.replace("json", "png")
            #img = cv2.cvtColor(cv2.imread(fn_png)) cv2.COLOR_BGR2RGB)
            img = cv2.imread(fn_png)

            # cut out face, left, right eye

            # 일단 눈 안 씀
            cropped = cut_face(img, eye)

            # also take 3D Gaze
            gaze3d = calculate_gaze3d(anno)
            # compute 2D gaze from... ? 
            gaze2d = calculate_gaze2d(anno)
            # save in Image and Label directory
            fn_org = fn_png.split("/home/hoseung/Work/NIA/")[1]
            fn_face = dir_out + f"Face/{cnt:06d}.jpg"
            fn_left = dir_out + f"Left/{cnt:06d}.jpg"
            fn_right = dir_out + f"Right/{cnt:06d}.jpg" 
            f.write(f"{fn_face} {fn_left} {fn_right} {fn_org} {gaze3d[0]},{gaze3d[1]},{gaze3d[2]} {gaze2d[0]},{gaze2d[1]}\n")

            cv2.imwrite(image_dir + fn_face, cropped)
        except:
            pass

        cnt +=1

    f.close()
    return cnt

In [175]:
cnt = make_dataset(jtrain, "train", 0)
cnt = make_dataset(jtest, "test", cnt)
cnt = make_dataset(jvalid, "val", cnt)
#cnt = make_dataset(jvalid, "val", cnt)
print("cnt", cnt)

cnt 3045
